# PFRA 

### _Development Notebook_
--- 

Tests using scrapbook

In [1]:
import sys;sys.path.append('../')

In [2]:
from hecrasio.core import *
from hecrasio.qaqc import *
import scrapbook as sb
import pandas as pd
import json
import itertools

#### Read model output data 

In [3]:
# Create a large list of notebooks
nbs = ["SB-QAQC-FluvialTest.ipynb" for i in range(10)]

#### Pull scraps

In [4]:
# Enumerate here to pull it into a dictionary
scraps = {str(i)+"_"+nb: list(sb.read_notebook(nb).scraps) for i, nb in enumerate(nbs)}

In [5]:
# Pull results into a dictionary of tables
table_dict = {nb: pd.concat([pd.read_json(result) for result in results_json]).T for nb, results_json in scraps.items()}

In [10]:
# Sample results
sample_nb_results = table_dict['0_SB-QAQC-FluvialTest.ipynb']
sample_nb_results

,Base Output Interval,Computation Time Courant Method,Computation Time Step Base,Computation Time Step Count To Double,Computation Time Step Max Courant,Computation Time Step Max Doubling,Computation Time Step Max Halving,Computation Time Step Min Courant,Flow Filename,Flow Title,...,Vol Accounting 2D Bdry,Vol Accounting Cell End,Vol Accounting Cell Start,Vol Accounting DS Outflow,Vol Accounting Error,Vol Accounting Error Percentage,Vol Accounting Groundwater,"Vol Accounting LS, Pump Div",Vol Accounting Lat Hydro,Vol Accounting US Inflow
Results,15MIN,Representative Length/Velocity,30SEC,2,2,2,1,0.5,DC_F01_NBR.u01,E0004,...,0,99553.8,84515.9,0,36.8438,0.00171806,0,0,0,0


In [13]:
sample_nb_results.columns

Index(['Base Output Interval', 'Computation Time Courant Method',
       'Computation Time Step Base', 'Computation Time Step Count To Double',
       'Computation Time Step Max Courant',
       'Computation Time Step Max Doubling',
       'Computation Time Step Max Halving',
       'Computation Time Step Min Courant', 'Flow Filename', 'Flow Title',
       'Geometry Filename', 'Geometry Title', 'Plan Filename', 'Plan Name',
       'Plan ShortID', 'Plan Title', 'Simulation End Time',
       'Simulation Start Time', 'Time Window', '1D Cores', '1D Methodology',
       '1D2D Flow Tolerance', '1D2D MaxIter', '1D2D Minimum Flow Tolerance',
       '1D2D WS Tolerance', '2D Boundary Condition Ramp Up Fraction',
       '2D Boundary Condition Volume Check', '2D Cores', '2D Coriolis',
       '2D Eddy Viscosity Transverse Mixing Coefficient', '2D Equation Set',
       '2D Initial Conditions Ramp Up Time (hrs)', '2D Latitude for Coriolis',
       '2D Maximum Iterations', '2D Names', '2D Number of Ti

In [28]:
#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-

# class QAQCResults:
#     """
#     Evaluates a results table.
#     """
    
#     def __init__(self)
    
def vol_acc_start_end(result_table:pd.core.frame.DataFrame):
    start = result_table['Vol Accounting Cell Start'][0]
    end = result_table['Vol Accounting Cell End'][0]
    if end > start:
        print("WARNING: End value greater than start value")

def vol_err(result_table:pd.core.frame.DataFrame, max_err:float):
    err_result = result_table['Vol Accounting Error'][0]
    if err_result > max_err:
        print("WARNING: Volume accounting error exceeded "+
              "with value of {}".format(err_result))

def vol_err_pct(result_table:pd.core.frame.DataFrame, max_err_pct:float):
    err_pct_result = result_table['Vol Accounting Error Percentage'][0]
    if err_pct_result > max_err_pct:
        print("WARNING: Volume accounting error percentage exceeded "+
              "with value of {}".format(err_result))

In [29]:
vol_err(sample_nb_results, 30)

# END